
### Running MongoDB

- exam version mongodb 3.4.9, pymongo 3.13.0 (found in gdrive A-level software)

- shortcut on Desktop (Default and ONLY way on exam machines)
- start menu MongoDB Server
    - (*default database is on user's profile AppData\Local\Temp*)
    - C:\data\db

- run command "mongod --dbpath=zzz", where zzz is a read/write accesible path
- run as a Windows Service

 *** Only 1 of the above can be used at any 1 time***


### Interacting with MongoDB with `pymongo`

Similar to relational databases, we need to know how to execute the important database operations (CRUD) with MongoDB as well. However, for MongoDB, we will skip on the MongoDB shell commands and go straight up to the commands in `pymongo`, which is a Python module to interact with MongoDB databases (as warned earlier, keep the MongoDB running else you will encounter errors.)

#### Connecting to MongoDB database with `pymongo`
To work with the database,
1. We first **establish connection** to the MongoDB server by creating `pymongo.MongoClient` object to `localhost` with the default port `27017`
2. Access the database through the client.
3. Access the collection through the database.
4. Perform  CRUD operations on the collection.

Working with database, collection and document from PyMongo:

*Tip* Everyting is a dictionary object or a list of dictionary objects


In [1]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)

print(f"These are the databases : {client.list_database_names()}")

# db = client.get_database("Overwatch")

# print(f"Collections in {db.name} are {db.list_collection_names()}" )


ModuleNotFoundError: No module named 'pymongo'

In [ ]:
## import the Overwatch database
## code will be discussed later
import json, pymongo
client = pymongo.MongoClient('localhost', 27017)
client.drop_database('Overwatch') ## cleanup previous stuff

f = open("Overwatch.Characters.json")
array_dict = json.load(f)

col = client['Overwatch']['Characters']
ret = col.insert_many(array_dict)
print(ret.acknowledged)

In [ ]:
## print all the documents in the Characters collection in the Overwatch DB
#col = client["Overwatch"].get_collection("Characters")
import pymongo
client = pymongo.MongoClient('localhost', 27017) # client obj
db = client.get_database("Overwatch") # database obj
col = db.get_collection("Characters") # collection obj

for d in col.find(): ## documents
    print(d)

In [ ]:
## Accessing the database and collection objects directly
import pymongo
client = pymongo.MongoClient('localhost', 27017) # client
list(client["school"]["teacher"].find())

#### CRUD operations with `pymongo`
Unlike `sqlite` which do CRUD operations by passing SQL statements into the `execute` command, the CRUD operations with `pymongo` is done through various methods to the objects found in `pymongo`. Some of the methods act on  `pymongo.collection.Collection` objects and they are:
- `insert_one()` : insert one document into a collection
- `insert_many()` : insert more than one document into a collection
- `find()` : to query documents from the collection
- `update_one()` : to update a document in the collection
- `update_many()` : to update more than one document in the collection
- `delete()`
- `drop()`

#### INSERT

In [ ]:
from pymongo import MongoClient



In [ ]:
list(col.find()) ## SELECT * FROM Person

In [ ]:
extra_persons = [
    {
    'name':'Ben',
    'age': '15',
    'hobbies': ['running','reading','gaming']
},{
    'name':'Lim Bo',
    'class': '18S01',
    'hobbies': ['gaming']
}
]

col.insert_many(extra_persons)

print(list(col.find()))

#### Exercise 2
load the students info in students.csv into the Person collection in test_info database

In [ ]:
import csv, pymongo
client = MongoClient('localhost', 27017)
col = client["test_info"]["Person"]
reader = csv.reader(open("students.csv"))
for student in list(reader)[1:]:
    print(student)
    col.insert_one(
        {
            "name2": student[2],
            "index_no": student[1],
            "class": student[0]
        }
    )


In [ ]:
list(col.find())

##### Example Using JSON

In [ ]:
import json, pymongo
client = pymongo.MongoClient('localhost', 27017)
client.drop_database('test_info') ## cleanup previous stuff

f = open("students.json")
d = json.load(f) # list of dictionary objects
col = client['test_info']['Person']
ret=col.insert_many(d)
print(ret.acknowledged)

In [ ]:
list(col.find())

____
#### Query Documents in a collection
```python
- <collection>.find(<query>, <projection>)  
- <collection>.find()
    #SELECT * FROM person

- <collection>.find({"name":"John"})
    #SELECT * FROM person WHERE name=  “John”
```
- query is a python dict with the attributes' values you want to retrieve
- result is a iterator of python dictionary objects
- `<collection>.find_one ({"name":"John"})` will return only 1 dictionary object

- Projection
    returns attributes define with a True/1 or 0/False to disable
```
db.person.find({ }, {"name":1,"_id:0})  
    SELECT name FROM person
```

In [ ]:
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person'] #client.db.collection
list( col.find( {'hobbies': 'running'},
                {'name':1, 'hobbies':1,
                 '_id':0}
               ) ) ## SELECT name, hobbies FROM Person WHERE 'running' IN hobbies

In [ ]:
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
col.find_one({
                "class" : "18S01"
             },
              { '_id':0})


#### Query a nested attribute

Example : Find all the students whose PM is Mr Chan

In [ ]:
## Code here
query =  {"PM.name": "Chan"}
projection = {"name":1, "class":1,"PM":1, "_id":0 }
list(col.find(query, projection))

In [ ]:
exp1 = { 'name': 'John'}
exp2 = { 'class': '18S02'}
query = { '$or' : [exp1, exp2]}
list(col.find( query, projection))

In [ ]:
list(col.find())

#### Complex Query: using operators in the query filter
```python
#implicit AND
<collection>.find({"name": "John", "class":"18S01"})
#SELECT * FROM person WHERE name=  “John” AND class = "18S01"

#explicit OR
<collection>.find( {"$or": [  { "name":"John"}, {"class":"18S02"  }  ] } )
#SELECT * FROM person WHERE name=  “John” OR class = "18S01"

```
<center>

| **Logical Operator** | **Description** |
|-|-|
| `$and` | Joins query clauses with a logical `AND` returns all documents that match the conditions of both clauses. |
| `$or ` | Joins query clauses with a logical OR returns all documents that match the conditions of either clause. |
| `$not ` | Inverts the effect of a query expression and returns documents that do not match the query expression. |

</center>


| **Comparison Operator** | **Description** |
|-|-|
| `$eq` | Matches values that are equal to the given value. |
| `$ne` | Matches values that are not equal to the given value. |
| `$gt` | Matches if values are greater than the given value. |
| `$gte` | Matches if values are greater or equal to the given value. |
| `$lt` | Matches if values are less than the given value. |
| `$lte` | Matches if values are less or equal to the given value. |
| `$in` | Matches any of the values in an array. |
| `$nin` | Matches none of the values specified in an array. |
| `$exists` | Check for existences of an attribute.|

</center>

Usage:
```python
query = { "age" : {"$gt": 30} } # age > 30
query = { "hobbies": {"$in": ["eating", "talking"]} } # hobbies attribute contains a value in array
query = { "hobbies" : {"$exists":0} } # hobbies attribute not in document

```

You can head to <a href='https://docs.mongodb.com/manual/reference/operator/query/'>the official query operator docs</a> for more examples and operators. However, the ones mentioned above should suffice for most cases.



In [ ]:
### Example: Find all PM's younger than 30
query =  {"PM.age":
          {'$gt': 30}
          }
projection = {"name":1,"PM.name":1, "PM.age":1, "_id":0 }
list(col.find(query, projection))

In [ ]:
## Using Python
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']

for doc in col.find({}, {'_id':0}):
    if doc["PM"]['age'] > 30 :
        print( doc["PM"]["name"], doc["PM"]["age"])


In [ ]:
## Composing a complex query:
## Find all students whose hobbies are chess OR kayaking
query_1 ={'hobbies':'chess'}
query_2 = {'hobbies':'kayaking'}
query = { "$or" : [ query_1, query_2] }
projection = {"name":1, "class":1,"hobbies":1, "_id":0 }
list(col.find(query, projection))

In [ ]:
## Find all students whose hobbies are chess OR kayaking using "$in" operators
exp = {"$in": ['chess', 'kayaking']}
query = {"hobbies": exp}
projection = {"name":1, "class":1,"hobbies":1, "_id":0 }
list(col.find(query, projection))

In [ ]:
## Find all students whose hobbies DOES NOT includes chess OR kayaking using "$not" operators
exp1 ={"$in": ['chess', 'kayaking']}
exp2 = {"$not": exp1}
query = {'hobbies':exp2}
projection = {"name":1, "hobbies":1, "_id":0 }
list(col.find(query, projection))
#print(list(col.find(query3, projection)))

In [ ]:
## Using Python :Find all students whose hobbies DOES NOT includes chess OR kayaking using "$not" operators
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
for doc in col.find({}, {'_id':0}):
    pass

### Exercise 3
In the `Person` collection in the `test_info` database,

1.  print out all the documents in the collection without showing the _id,

2.  find the documents with 'class' not equal '18S01`

3.  find the documents whose class is 18S01 and PM's age is > 30

4.  find all student names whose has gaming as a hobby

5.   find all student names who has gaming or bowling as a hobby

6.  find all student names who do not have any hobby

In [ ]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)
db = client.get_database("test_info")
col = db.get_collection("Person")

In [ ]:
## Same as above, more compact form
import pymongo
col = pymongo.MongoClient('localhost', 27017)["test_info"]["Person"]

In [ ]:
#Task 1
projection = {"_id":0, "name":True}
print(list(
        col.find({}, projection)
        )
    )

In [ ]:
#Task 2
projection = {"_id":0}
query1 = {'class':{'$ne':'18S01'}}
print(list(col.find(query1, projection)))

In [ ]:
## Pythonic
for doc in col.find():
    if doc.get('class') != '18S01':
        print(f"{doc['name']},{doc['class']}")

In [ ]:
# Task 3
projection = {"_id":0,'class':1,"PM":1}
query1 = {'class': '18S01'}
query2 = {'PM.age': {'$gt':30}}
query3 = {'$and': [query1, query2]}
query4 = {
    'class': '18S01',
    'PM.age': {'$gt':30}
}
print(list(col.find(query4, projection)))

In [ ]:
## Pythonic
for doc in col.find():
    if doc['class'] == '18S01' and doc["PM"]["age"] > 30:
        print(f"{doc['class']},{doc['PM']['name']},{doc['PM']['age']}")

In [ ]:
# Task 4
projection = {'name':1,'_id':0,'hobbies':1}
query= {'hobbies': 'gaming'}

print(list(col.find(query, projection)))

In [ ]:
## Pythonic
for doc in col.find():
    if doc.get("hobbies")  and   "gaming" in doc["hobbies"]:
        print( doc["name"], doc["hobbies"])

In [ ]:
# Task 5
projection = {'name':1,'_id':0, 'hobbies':1}
exp = {'$in':['gaming','bowling'] }
query1 = {'hobbies': exp}
print(list(col.find(query1, projection)))

In [ ]:
## Pythonic
for doc in col.find():
    if doc.get("hobbies") and ( "bowling" in doc["hobbies"] or "gaming" in doc["hobbies"]):
        print( doc["name"], doc["hobbies"])

In [ ]:
# Task 6
projection = {'name':1,'hobbies': 1 ,'_id':0}
query1 = {'hobbies':{'$exists':0}}
list(
    col.find(query1, projection)
)

In [ ]:
for doc in col.find():
    if not doc.get('hobbies'):
        print(doc['name'])

#### Updating Documents in a collection [UPDATE]

1. Update one document

    - use `<collection>.update_one(<query>,<update>)`, where
    ```python
    <update> :=
    {'$set':
        {
            <attribute_name_1>:<value_1>,
            <attribute_name_2>:<value_2>,...
        }
    }
    ```
    > Take note that if an attribute does not exits in the current document, the attribute, with its value will be created
    
2. Update many documents

    - update all documents satisfying your query, use `<collection>.update_many(<query>,<update>)`.

3. `update_one()` and `update_many()` methods accept a Boolean parameter `upsert` (default is `False`) which modify their behaviours a little bit. If `upsert=True` and the query does not match any documents in MongoDB, the method will perform an insertion of the document instead. Otherwise, if the query does not match any documents, no insertion of such records. This approach could be handy if you want to avoid using conditionals to handle such cases.

Example:
```
db.person.updateOne({"name":"John"},
                    {"$set":{
                        "name": "NewJohn",
                        "class":"19S01"
                        }
                    } )
# UPDATE person SET name =“NewJohn”,class="19S01"  WHERE name=“John”
```
### Exercise 4
In the `Person` collection in the `test_info` database, update all the documents with PM.name = "Chan"  set the PM.age field for such documents to `50`.

In [ ]:
#YOUR_CODE_HERE
# print(list(coll.find()))
import pymongo
col = pymongo.MongoClient('localhost', 27017)["test_info"]["Person"]
project = {"_id":0, "PM":1}
query = { "PM.name": "Chan"}
update = {"$set":
            {
               "PM.age":50
            }
         }

#col.update_many(query, update)

print(list(col.find(query, project)))

In [ ]:
##Pythonic
for doc in col.find():
    if doc["PM"]["name"] == "Chan":
        col.update_one(
            {"_id": doc["_id"]},
            {"$set":
                {
                    "PM.age": 34
                }
             }
        )

#### Delete Documents in a collection [DELETE]

To:
- delete one document satisfying your query, we use `coll.delete_one(<my_query>)`
- delete all documents satisfying your query, we use `coll.delete_many(<my_query>)`
- delete all documents in the collection, we pass empty query `{}` in the `coll.delete_many()` method.

### Exercise 5
In the `Person` collection in the `test_info` database,

- delete all documents from `test_info` database and verify that the database is empty.
- remove the Person collection

In [ ]:
#YOUR_CODE_HERE

#col.delete_many({})

# print(list(col.find()))
col.drop()

## Exercise 23.1 2021/DHS/P2/Q4 H2 Computing

### Task 1
Write a program to help staff of an events company to insert data into a NoSQL database products under the collection balloons.

The data is provided for you in `balloons.json` as well as in the table below where the first row are headers for the fields.

<center>

| `design` | `amount` | `helium` | `colours` |
|-|-|-|-|
| car | 88 | no | red, yellow |
| cloud | 14 | | blue, green |
| flower | 75 | yes | red, blue |
| bag | 38 | no | red, blue, black |

</center>

Each colour in `colours` field should be an item in an array. <div style="text-align: right">[6]</div>

In [ ]:
#YOUR_CODE_HERE
import json, pymongo
data = json.load(open('balloons.json'))
col = pymongo.MongoClient('localhost',27017)['products']['balloons']
col.drop()
col.insert_many(data).acknowledged


In [ ]:
list(col.find())

### Task 2
Write code to print the `amount` of the product with the design "`car`". <div style="text-align: right">[2]</div>

In [ ]:
#YOUR_CODE_HERE
query = {
    "design": "car"
}
for d in col.find():
    if d["design"]== "car":
        print(d["amount"])

### Task 3
Write code to update the field helium to have the value "no" for all documents which do not have a field or value(None) for helium. <div style="text-align: right">[3]</div>

In [ ]:
#YOUR_CODE_HERE
query1 = {'helium':None}
query2 = {"helium": {'$exists':0} }
query3 = {'$or':[query1, query2]}
ret = col.update_many(query3, {'$set':{'helium':'no'}} )

In [ ]:
#YOUR_CODE_HERE
# {"helium": None}, no helium attribute
for d in col.find():
    if d.get("helium") == None:
        col.update_one(
            {"_id": d["_id"]},
            {"$set":
                {
                    "helium": "no"
                }
             }
        )
